<a href="https://colab.research.google.com/github/LukasRodriguess/process_automation/blob/main/Automatiza%C3%A7%C3%A3o_de_processos_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Automatização de processos

---

A ideia desse projeto é tornar operacional um processo antes executado manualmente,  utilizando para isso da tecnologia e da integração de sistemas e dados.

Estarei utilizando Python, Requets, SQLite


Coisas para melhorar no projeto:


1.   Adicionar ID automatico
2.   ajustar end e start routine para no final da execução sair só o tempo sem a data
3. passar todo o projeto para inglês
4. buscar melhorias e implementações




##### Libraries ↓

In [1]:
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import pandas_datareader.data as pdr
import yfinance as yf #API Yahoo finance

import sqlite3

#libs for the operating system ↓
import os
import platform
#import win32com.client as win32

#libs for connection ↓
import requests
import socket

#libs to ignore warnings ↓
import warnings
warnings.filterwarnings('ignore')



#####Rotinas Iniciais

In [12]:
routine_id = 1
routine_name = 'Fluxo de dados - Corretora'

#Identify user information
def identify_user():
  user = os.environ.get('USERNAME') #Catch user name
  machine = platform.node() #Catch machine name
  operating_system = platform.platform() #Catch operating System

  return (user, machine, operating_system)

#---------------------------------------------------------------------------------
#Identify the beginning of the routine
def start_routine():
  date_start = datetime.today().date()
  hour_start = datetime.now()

  return (date_start, hour_start)

#---------------------------------------------------------------------------------
#identify the end of the routine
def end_routine():
  date_end = datetime.today().date()
  hour_end = datetime.now()

  return (date_end, hour_end)

#---------------------------------------------------------------------------------
def temp_exe():
    calc = list_end[1] - list_start[1]
    # Convert timedelta to datetime
    calc_datetime = datetime.datetime(1, 1, 1) + calc

    # Format the datetime object
    hour_min_sec = calc_datetime.strftime("%H:%M:%S")

    return hour_min_sec

#---------------------------------------------------------------------------------
def check_connection():
  URL = "https://www.google.com"

  time_out = 5
  try:
    requests.get(URL, timeout=time_out)
    return True

  except:
    return False

#---------------------------------------------------------------------------------
def identify_ip():
  try:
    local_ip = socket.gethostbyname(socket.gethostname())
    return local_ip
  except:
    return False


#---------------------------------------------------------------------------------
# Gerando os parametros
list_user = identify_user()
list_start = start_routine()
connection_net = check_connection()
check_ip = identify_ip()

# Colocar todos os erros que acontecer nosso processo
operational_error = ''

#####Operacional

In [ ]:
try:
  yf.pdr_override() #core
  data_base = pdr.get_data_yahoo('ARZZ3.SA', start='2024-01-01') #get data
  eixo_plot = data_base['Close']

  #Configurando grafico ↓
  plt.figure( figsize=(15,6) ) #tamanho
  plt.style.use('ggplot') #estilo

  plt.plot(eixo_plot) #plot do grafico

  #labels e titulos ↓
  plt.title("Analise Arezzo") #title
  plt.xlabel('Periodo') #Eixo X
  plt.ylabel('Preço fechamento')#Eixo y

  plt.savefig('Indicador_Ações.png') #salvando grafico em png

except:
  operational_error = 'Execution_Error'

# Termino da Rotina
list_end = end_routine()

##### Organização

In [ ]:
# Organização
dictionary = {
    'Id_Routine': routine_id,
    'Name_Routine': routine_name,
    'User': list_user[0],
    'Machine': list_user[1],
    'Operation_System': list_user[2],
    'Date_Start': list_start[0],
    'Hour_Start': list_start[1],
    'Try_connection': connection_net,
    'Local_Ip': check_ip,
    'Date_end': list_end[0],
    'Hour_end': list_end[1],
    'Timex_Ex': list_end[1] - list_start[1],
    'Error': operational_error
}

#log table
log_table = pd.DataFrame(dictionary, index=[0])


#Connect Data Base ↓
conn_db = sqlite3.connect('DataBase.db')
cursor = conn_db.cursor()

log_table.to_sql(

    #Nome da  tabela
    'process_table',

    #conexão do banco
    conn_db,

    #se a tabela existe
    if_exists='append',

    #ignorar o index para nao dar conflito com o index do pandas
    index=False
)


In [ ]:
pd.read_sql('SELECT * FROM process_table', conn_db)

In [7]:
conn_db.close()

#Glossary

yf.pdr_override() #pesquisar sobre





```
#Codigo utilizado para criar o banco e sua tabela de registro ↓

import sqlite3
conn_db = sqlite3.connect('DataBase.db')
cursor = conn_db.cursor()

cursor.execute("""
CREATE TABLE process_table (
    Id_Routine TEXT,
    Name_Routine TEXT,
    User TEXT,
    Machine TEXT,
    Operation_System TEXT,
    Date_Start TEXT,
    Hour_Start TEXT,
    Try_connection TEXT,
    Local_Ip TEXT,
    Date_end TEXT,
    Hour_end TEXT,
    Timex_Ex TEXT,
    Error TEXT
)
""")
conn_db.commit()
conn_db.close()
```

